In [4]:
import gzip
import pandas as pd
import numpy as np
from mnist import MNIST
import torch
from torch import nn
import copy
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary

In [7]:
class Mish(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x = x * torch.tanh(F.softplus(x))
        return x

In [6]:
class Conv_Layer_box(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size, stride, activation_func, batch_normalization):
        super().__init__()
        padding = (int((kernel_size - 1)/2), int((kernel_size - 1)/2))
        
        dict_activation_func = {"ReLU": nn.ReLU(inplace=True),
                                "Leaky": nn.LeakyReLU(0.1, inplace=True),
                                "Mish": Mish()
                               }
        
        if batch_normalization == True:
            bias = False
        else:
            bias = True
        self.conv_box = nn.ModuleList()
        self.conv_box.append(nn.Conv2d(in_channel, out_channel, kernel_size = kernel_size, stride = stride, padding = padding, bias = bias))
        if batch_normalization == True:
            self.conv_box.append(nn.BatchNorm2d(out_channel))
        self.conv_box.append(dict_activation_func[activation_func])
        
    def forward(self, x):
        for layer in self.conv_box:
            x = layer(x)
        return x
            
class RSL_Layer_box(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size, stride, activation_func, batch_normalization, loop_time):
        super().__init__()
        self.RSL_box = nn.ModuleList()
        
        for k in range(loop_time):
            for i in range(len(in_channel)):
                self.RSL_box.append(Conv_Layer_box(in_channel[i], out_channel[i], kernel_size= kernel_size[i], stride = stride[i], activation_func = activation_func[i], batch_normalization = batch_normalization[i]))
    
    def forward(self, x):
        counting = 0
        shortcut_value = x
        for layer in self.conv_box:
            x = layer(x)
            counting += 1 
            if counting == len(in_channel):
                x = x + shortcut_value
                shortcut_value = x
                counting = 0
            
        return x

class Maxpool_pad_Layer_box(nn.Module):
    def __init__(self, maxpool_size):
        super().__init__()
        self.maxpool_size = maxpool_size
        #why there are 2 padding??????????????
        self.pad_1 = (self.maxpool_size - 1) / 2
        self.pad_2 = self.maxpool_size - self.pad_1
    def forward(self, x):
        x = F.pad(x, (self.padding1, self.padding2, self.padding1, self.padding2), mode='replicate')
        x = F.max_pool2d(x, self.maxpool_size, stride=1)
        return x
class Upsample_layer(nn.Module):
    def __init__(self, stride):
        super().__init__()
        self.stride = stride
        
    def forward(self, x):
        batch, channel, height, width = x.data.size()
        x = x.view(batch, channel, height, 1, width, 1).expand(batch, channel, height, self.stride, width, self.stride)
        x = x.contiguous().view(batch, channel, height * self.stride, width * self.stride)
        return x

In [3]:
class YOLO_v4_1st_downsampling_Layer_group(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer_0 = Conv_Layer_box(3, 32, kernel_size=3, stride = 1, activation_func = "Mish", batch_normalization = True)
        self.conv_layer_1 = Conv_Layer_box(32, 64, kernel_size=3, stride = 2, activation_func = "Mish", batch_normalization = True)
        self.conv_layer_2 = Conv_Layer_box(64, 64, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)

        #route - name - 3
        self.conv_layer_4 = Conv_Layer_box(64, 64, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        self.conv_layer_5 = Conv_Layer_box(64, 32, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        self.conv_layer_6 = Conv_Layer_box(32, 64, kernel_size=3, stride = 1, activation_func = "Mish", batch_normalization = True)
        #shortcut?????? - name - 7
        self.conv_layer_8 = Conv_Layer_box(64, 64, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        #route - name - 9
        #self.conv_layer_9 = Conv_Layer_box(64, 128, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        
    def forward(self, x):
        out_0 = self.conv_layer_0(x)
        out_1 = self.conv_layer_1(out_0)
        out_4 = self.conv_layer_4(out_1)
        out_5 = self.conv_layer_5(out_4)
        out_6 = self.conv_layer_6(out_5)
        shortcut_7 = out_6 + out_1
        out_8 = self.conv_layer_8(shortcut_7)
        out_2 = self.conv_layer_2(out_1)
        route_9 = out_8 + out_2
        #out_9 = self.conv_layer_9(route_9)
        out = route_9
        return out

In [3]:
class YOLO_v4_2nd_downsampling_Layer_group(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer_10 = Conv_Layer_box(128, 64, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        self.conv_layer_11 = Conv_Layer_box(64, 128, kernel_size=3, stride = 2, activation_func = "Mish", batch_normalization = True)
        self.conv_layer_12 = Conv_Layer_box(128, 64, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        #route - name - 13
        self.conv_layer_14 = Conv_Layer_box(128, 64, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        self.RSL_layer = RSL_Layer_box(in_channel = [64, 64], out_channel = [64, 64], kernel_size = [1, 3], stride = [1, 1], activation_func = ["Mish", "Mish"], batch_normalization = [True, True], loop_time = 2))
        self.conv_layer_21 = Conv_Layer_box(64, 64, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        #self.conv_layer_22 = Conv_Layer_box(64, 128, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)

        
    def forward(self, x):
        out_10 = self.conv_layer_10(x)
        out_11 = self.conv_layer_11(out_10)
        out_14 = self.conv_layer_14(out_11)
        shortcut_20 = self.RSL_layer(out_14)
        out_21 = self.conv_layer_21(shortcut_20)
        out_12 = self.conv_layer_12(out_11)
        route_22 = out_21 + out_12
        #out_22 = self.conv_layer_22(route_22)
        out = route_22
        return out

In [4]:
class YOLO_v4_3rd_downsampling_Layer_group(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer_23 = Conv_Layer_box(128, 128, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        self.conv_layer_24 = Conv_Layer_box(128, 256, kernel_size=3, stride = 2, activation_func = "Mish", batch_normalization = True)
        
        self.conv_layer_25 = Conv_Layer_box(256, 128, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        #route - name - 26
        self.conv_layer_27 = Conv_Layer_box(256, 128, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        self.RSL_layer = RSL_Layer_box(in_channel = [128, 128], out_channel = [128, 128], kernel_size = [1, 3], stride = [1, 1], activation_func = ["Mish", "Mish"], batch_normalization = [True, True], loop_time = 8))
        self.conv_layer_52 = Conv_Layer_box(128, 128, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        #self.conv_layer_53 = Conv_Layer_box(128, 256, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        self.conv_layer_54 = Conv_Layer_box(256, 256, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        
        
    def forward(self, x):
        out_23 = self.conv_layer_23(x)
        out_24 = self.conv_layer_24(out_23)
        out_27 = self.conv_layer_27(out_24)
        shortcut_51 = self.RSL_layer(out_27)
        out_52 = self.conv_layer_52(shortcut_51)
        out_25 = self.conv_layer_25(out_24)
        route_53 = out_52 + out_25
        #out_53 = self.conv_layer_53(route_53)
        out_54 = self.conv_layer_54(route_53)
        out = out_54
        return out

In [5]:
class YOLO_v4_4th_downsampling_Layer_group(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer_55 = Conv_Layer_box(256, 512, kernel_size=3, stride = 2, activation_func = "Mish", batch_normalization = True)
        
        self.conv_layer_56 = Conv_Layer_box(512, 256, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        #route - name - 57
        self.conv_layer_58 = Conv_Layer_box(512, 256, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        self.RSL_layer = RSL_Layer_box(in_channel = [256, 256], out_channel = [256, 256], kernel_size = [1, 3], stride = [1, 1], activation_func = ["Mish", "Mish"], batch_normalization = [True, True], loop_time = 8))
        self.conv_layer_83 = Conv_Layer_box(256, 256, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        #self.conv_layer_84 = Conv_Layer_box(256, 512, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        self.conv_layer_85 = Conv_Layer_box(512, 512, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        
        
    def forward(self, x):
        
        out_55 = self.conv_layer_55(out_54)
        out_58 = self.conv_layer_58(out_55)
        shortcut_82 = self.RSL_layer(out_58)
        out_83 = self.conv_layer_83(shortcut_82)
        out_56 = self.conv_layer_56(out_55)
        route_84 = out_83 + out_56
        #out_84 = self.conv_layer_84(route_84)
        out_85 = self.conv_layer_85(route_84)
        out = out_84
        return out

In [6]:
class YOLO_v4_5th_downsampling_Layer_group(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer_86 = Conv_Layer_box(512, 1024, kernel_size=3, stride = 2, activation_func = "Mish", batch_normalization = True)
        
        self.conv_layer_87 = Conv_Layer_box(1024, 512, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        #route - name - 88
        self.conv_layer_89 = Conv_Layer_box(1024, 512, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        self.RSL_layer = RSL_Layer_box(in_channel = [512, 512], out_channel = [512, 512], kernel_size = [1, 3], stride = [1, 1], activation_func = ["Mish", "Mish"], batch_normalization = [True, True], loop_time = 4))
        self.conv_layer_102 = Conv_Layer_box(512, 512, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        self.conv_layer_103 = Conv_Layer_box(512, 1024, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        self.conv_layer_104 = Conv_Layer_box(1024, 1024, kernel_size=1, stride = 1, activation_func = "Mish", batch_normalization = True)
        
    def forward(self, x):
        
        out_86 = self.conv_layer_86(x)
        out_89 = self.conv_layer_89(out_86)
        shortcut_101 = self.RSL_layer(out_89)
        out_102 = self.conv_layer_102(shortcut_101)
        out_87 = self.conv_layer_87(out_86)
        route_103 = out_102 + out_87
        out_103 = self.conv_layer_103(route_103)
        out_104 = self.conv_layer_104(out_103)
        out = out_104
        return out

In [1]:
class YOLO_v4_Neck(nn.Module):
    def __init__(self):
        super().__init__()

        
        self.conv_layer_105 = Conv_Layer_box(1024, 512, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_106 = Conv_Layer_box(512, 1024, kernel_size=3, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_107 = Conv_Layer_box(1024, 512, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        
        self.Maxpool_pad_Layer_108 = Maxpool_pad_Layer_box(5)
        self.Maxpool_pad_Layer_110 = Maxpool_pad_Layer_box(9)
        self.Maxpool_pad_Layer_112 = Maxpool_pad_Layer_box(13)
        
        
        
        #self.conv_layer_113 = Conv_Layer_box(512, 2048, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_114 = Conv_Layer_box(512, 512, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_115 = Conv_Layer_box(512, 1024, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_116 = Conv_Layer_box(1024, 512, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_117 = Conv_Layer_box(512, 256, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        
        self.Upsample_layer_118 = Upsample_layer(stride = 2)
        
        self.conv_layer_120 = Conv_Layer_box(512, 256, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        
        self.conv_layer_122 = Conv_Layer_box(256, 256, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_123 = Conv_Layer_box(256, 512, kernel_size=3, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_124 = Conv_Layer_box(512, 256, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        
        self.conv_layer_125 = Conv_Layer_box(256, 512, kernel_size=3, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_126 = Conv_Layer_box(512, 256, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_127 = Conv_Layer_box(256, 128, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        
        self.Upsample_layer_128 = Upsample_layer(stride = 2)
        
        self.conv_layer_130 = Conv_Layer_box(256, 128, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        
        self.conv_layer_132 = Conv_Layer_box(128, 128, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_133 = Conv_Layer_box(128, 256, kernel_size=3, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_134 = Conv_Layer_box(256, 128, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_135 = Conv_Layer_box(128, 256, kernel_size=3, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_136 = Conv_Layer_box(256, 128, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        
        
        
        
        
    def forward(self, layer_105_in, layer_120_in, layer_130_in):
        out_105 = self.conv_layer_105(layer_105_in)
        out_106 = self.conv_layer_106(out_105)
        out_107 = self.conv_layer_107(out_106)
        
        out_108 = Maxpool_pad_Layer_108(out_107)
        out_110 = Maxpool_pad_Layer_108(out_107)
        out_112 = Maxpool_pad_Layer_108(out_107)
        
        route_113 = out_107 + out_108 + out_110 + out_112
        
        out_114 = conv_layer_114(out_113)
        out_115 = conv_layer_115(out_114)
        out_116 = conv_layer_116(out_115)
        out_117 = conv_layer_117(out_116)
        
        out_118 = Upsample_layer_118(out_117)
        
        out_120 = conv_layer_120(layer_120_in)
        
        route_121 = out_118 + out_120
        
        
        out_122 = conv_layer_122(layer_121_in)
        out_123 = conv_layer_123(layer_122_in)
        out_124 = conv_layer_124(layer_123_in)
        out_125 = conv_layer_125(layer_124_in)
        out_126 = conv_layer_126(layer_125_in)
        out_127 = conv_layer_127(layer_126_in)
        
        out_128 = Upsample_layer_128(out_127)
        
        out_130 = conv_layer_130(layer_130_in)
        
        route_131 = out_128 + out_130
        
        
        out_132 = conv_layer_132(layer_131_in)
        out_133 = conv_layer_133(layer_132_in)
        out_134 = conv_layer_134(layer_133_in)
        out_135 = conv_layer_135(layer_134_in)
        out_136 = conv_layer_136(layer_135_in)
        
        
        
        
        return out_136, out_126, out_116

NameError: name 'nn' is not defined

In [ ]:
class YOLO_v4_head(nn.Module):
    def __init__(self):
        super().__init__()
        
        
        self.conv_layer_137 = Conv_Layer_box(128, 256, kernel_size=3, stride = 1, activation_func = "Leaky", batch_normalization = True)
        #pending: Linear activation function?
        self.conv_layer_138 = Conv_Layer_box(256, 255, kernel_size=1, stride = 1, activation_func = "ReLU", batch_normalization = True)
        #139 - YOLO layer
        
        
        self.conv_layer_141 = Conv_Layer_box(128, 256, kernel_size=3, stride = 2, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_143 = Conv_Layer_box(256, 256, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_144 = Conv_Layer_box(256, 512, kernel_size=3, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_145 = Conv_Layer_box(512, 256, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_146 = Conv_Layer_box(256, 512, kernel_size=3, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_147 = Conv_Layer_box(512, 256, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_148 = Conv_Layer_box(256, 512, kernel_size=3, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_149 = Conv_Layer_box(512, 255, kernel_size=1, stride = 1, activation_func = "ReLU", batch_normalization = True)
        #150 - YOLO layer
        self.conv_layer_152 = Conv_Layer_box(256, 512, kernel_size=3, stride = 2, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_154 = Conv_Layer_box(512, 512, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_155 = Conv_Layer_box(512, 1024, kernel_size=3, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_156 = Conv_Layer_box(1024, 512, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_157 = Conv_Layer_box(512, 1024, kernel_size=3, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_158 = Conv_Layer_box(1024, 512, kernel_size=1, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_159 = Conv_Layer_box(512, 1024, kernel_size=3, stride = 1, activation_func = "Leaky", batch_normalization = True)
        self.conv_layer_160 = Conv_Layer_box(1024, 255, kernel_size=1, stride = 1, activation_func = "ReLU", batch_normalization = True)
        #161 - YOLO layer
        
    def forward(self, layer_136_out, layer_126_out, layer_116_out):
        out_137 = self.conv_layer_137(layer_136_out)
        out_138 = self.conv_layer_138(out_137)
        #139 - YOLO layer
        
        out_141 = self.conv_layer_141(layer_136_out)
        route_142 = out_141 + layer_126_out
        out_143 = self.conv_layer_143(route_142)
        out_144 = self.conv_layer_144(out_143)
        
        out_145 = self.conv_layer_145(out_144)
        out_146 = self.conv_layer_146(out_145)
        out_147 = self.conv_layer_147(out_146)
        out_148 = self.conv_layer_148(out_147)
        
        out_149 = self.conv_layer_149(out_148)
        #150 - YOLO layer
        
        out_152 = self.conv_layer_152(out_147)
        route_153 = out_152 + layer_116_out
        
        
        out_154 = self.conv_layer_154(route_153)
        
        out_155 = self.conv_layer_155(out_154)
        out_156 = self.conv_layer_156(out_155)
        out_157 = self.conv_layer_157(out_156)
        out_158 = self.conv_layer_158(out_157)
        
        out_159 = self.conv_layer_159(out_158)
        
        
        out_160 = self.conv_layer_160(out_159)
        #161 - YOLO layer
        
        return ???

In [ ]:
class YOLO_v4_Module(nn.Module):
    def __init__(self):
        super(YOLO_v4_Module, self).__init__()
        #self.input_size = input_size
        #self.output_size = output_size

        
        self.downsample_layer_1 = nn.Sequential(
            nn.Identity()
            nn.Conv2d(64, 32, kernel_size=1),
            nn.Conv2d(32, 64, kernel_size=3),
            
        )
        
        self.conv_layer_2 = nn.Conv2d(192, 128, kernel_size=1)
        
        self.conv_layer_3 = nn.Conv2d(192, 128, kernel_size=1)
        self.conv_layer_4 = nn.Conv2d(128, 256, kernel_size=3, padding=(1,1))
        self.conv_layer_5 = nn.Conv2d(256, 256, kernel_size=1)
        self.layer_6 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=1),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.conv_layer_7 = nn.Conv2d(512, 256, kernel_size=1)
        self.conv_layer_8 = nn.Conv2d(256, 512, kernel_size=3, padding=(1,1))
        self.conv_layer_9 = nn.Conv2d(512, 256, kernel_size=1)
        self.conv_layer_10 = nn.Conv2d(256, 512, kernel_size=3, padding=(1,1))
        self.conv_layer_11 = nn.Conv2d(512, 256, kernel_size=1)
        self.conv_layer_12 = nn.Conv2d(256, 512, kernel_size=3, padding=(1,1))
        self.conv_layer_13 = nn.Conv2d(512, 256, kernel_size=1)
        self.conv_layer_14 = nn.Conv2d(256, 512, kernel_size=3, padding=(1,1))
        self.conv_layer_15 = nn.Conv2d(512, 512, kernel_size=1)
        self.layer_16 = nn.Sequential(
            nn.Conv2d(512, 1024, kernel_size=3, padding=(1,1)),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.conv_layer_17 = nn.Conv2d(1024, 512, kernel_size=1)
        self.conv_layer_18 = nn.Conv2d(512, 1024, kernel_size=3, padding=(1,1))
        self.conv_layer_19 = nn.Conv2d(1024, 512, kernel_size=1)
        self.conv_layer_20 = nn.Conv2d(512, 1024, kernel_size=3, padding=(1,1))
        self.conv_layer_21 = nn.Conv2d(1024, 1024, kernel_size=3, padding=(1,1))
        self.conv_layer_22 = nn.Conv2d(1024, 1024, kernel_size=3, stride=2, padding=(1,1))
        self.conv_layer_23 = nn.Conv2d(1024, 1024, kernel_size=3, padding=(1,1))
        self.conv_layer_24 = nn.Conv2d(1024, 1024, kernel_size=3, padding=(1,1))
        
        self.fc1 = nn.Linear(7 * 7 * 1024, 4096)
        self.fc2 = nn.Linear(4096, 7 * 7 * 30)
    def forward(self, x):
        out = self.layer_1(x)
        out = self.layer_2(out)
        out = self.conv_layer_3(out)
        out = self.conv_layer_4(out)
        out = self.conv_layer_5(out)
        out = self.layer_6(out)
        out = self.conv_layer_7(out)
        out = self.conv_layer_8(out)
        out = self.conv_layer_9(out)
        out = self.conv_layer_10(out)
        out = self.conv_layer_11(out)
        out = self.conv_layer_12(out)
        out = self.conv_layer_13(out)
        out = self.conv_layer_14(out)
        out = self.conv_layer_15(out)
        out = self.layer_16(out)
        out = self.conv_layer_17(out)
        out = self.conv_layer_18(out)
        out = self.conv_layer_19(out)
        out = self.conv_layer_20(out)
        out = self.conv_layer_21(out)
        out = self.conv_layer_22(out)
        out = self.conv_layer_23(out)
        out = self.conv_layer_24(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out